Updated: 02/02/24

This workbook follows Cheung et al. 2023 and attempts to reproduce the main results. In particular, by using free frequency fitting to select both linear and possible nonlinear modes present in a particular (l,m). 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import qnmfitsrd as qnmfits
from multiprocessing import Pool
from spatial_reconstruction_tests.spatial_reconstruction import *
from matplotlib.animation import FuncAnimation
from scipy.interpolate import UnivariateSpline
from Visualisation.qnm_vis import * 
from qnmfitsrd.CCE_file_getter import *
import datetime
from scipy.optimize import least_squares

l_max = 5
n_max = 5

In [3]:
CCE_sim1 = CCE_to_sim_simplified(sim_num='0001')
CCE_sim2 = CCE_to_sim_simplified(sim_num='0002')
CCE_sim3 = CCE_to_sim_simplified(sim_num='0003')
CCE_sim4 = CCE_to_sim_simplified(sim_num='0004')
CCE_sim5 = CCE_to_sim_simplified(sim_num='0005')
CCE_sim6 = CCE_to_sim_simplified(sim_num='0006')
CCE_sim7 = CCE_to_sim_simplified(sim_num='0007')
CCE_sim8 = CCE_to_sim_simplified(sim_num='0008')
CCE_sim9 = CCE_to_sim_simplified(sim_num='0009')
CCE_sim10 = CCE_to_sim_simplified(sim_num='0010')
CCE_sim11 = CCE_to_sim_simplified(sim_num='0011')
CCE_sim12 = CCE_to_sim_simplified(sim_num='0012')
CCE_sim13 = CCE_to_sim_simplified(sim_num='0013')

CCE_sim305 = CCE_to_sim_simplified(sim_num='0305')

In [ ]:
sim_1502 = qnmfits.SXS(ID=1502, zero_time=(2,2))
sim_1476 = qnmfits.SXS(ID=1476, zero_time=(2,2))
sim_1506 = qnmfits.SXS(ID=1506, zero_time=(2,2))
sim_1508 = qnmfits.SXS(ID=1508, zero_time=(2,2))
sim_1474 = qnmfits.SXS(ID=1474, zero_time=(2,2))
sim_1505 = qnmfits.SXS(ID=1505, zero_time=(2,2))
sim_1504 = qnmfits.SXS(ID=1504, zero_time=(2,2))
sim_1485 = qnmfits.SXS(ID=1485, zero_time=(2,2))
sim_1486 = qnmfits.SXS(ID=1486, zero_time=(2,2))
sim_1441 = qnmfits.SXS(ID=1441, zero_time=(2,2))
sim_1500 = qnmfits.SXS(ID=1500, zero_time=(2,2))
sim_1492 = qnmfits.SXS(ID=1492, zero_time=(2,2))
sim_1465 = qnmfits.SXS(ID=1465, zero_time=(2,2))
sim_1458 = qnmfits.SXS(ID=1458, zero_time=(2,2))
sim_1438 = qnmfits.SXS(ID=1438, zero_time=(2,2))
sim_1430 = qnmfits.SXS(ID=1430, zero_time=(2,2))

# Reproducing main result 

Iterate through all qnms up to some l_max, n_max. For a given QNM, allow the frequency to vary and fit for the frequency that minimises mismatch. If this frequency is consistent with prediction and doesn't vary over a range of $t_0$, add it to the model. Repeat for the next QNM to build up a full linear model of QNMs present in a particular (l,m). 


Note: should try randomising this to eliminate bias? 

In [14]:
sim = CCE_sim305

start_times = np.linspace(-40, 40, 100)

"""
QNMs = [(lam,mu,n,p) for lam in np.arange(2, l_max+1)
                        for mu in np.arange(-lam, lam+1)
                           for n in np.arange(0, 1)
                              for p in (-1, +1)]
"""

QNMs = [(2,2,0,1), (4,4,0,1), (5,4,0,1)]

current_model = [] 

for QNM in QNMs:
    new_model = current_model + [QNM]
    omegas = [] 
    for t0 in start_times:
        omega_bestfit = qnmfits.free_frequency_fit(sim.times, sim.h[4,4], t0, modes=new_model, Mf=sim.Mf, chif=sim.chif_mag)
        omegas.append(omega_bestfit)
        #plt.scatter(omega_bestfit.real, omega_bestfit.imag)
    #plt.title('New model: ' + str(new_model))
    #plt.show() 
    omega_0 = omegas[40]
    frequency = np.array(qnm.omega_list([QNM], sim.chif_mag, sim.Mf))
    print(frequency)
    print(omega_0)
    include_bool = np.isclose(omega_0.real, frequency.real, rtol=1e-01) & np.isclose(omega_0.imag, frequency.imag, rtol=1e-01)
    if include_bool == 'y':
        current_model = new_model
    else:
        continue


[0.55578191-0.08517235j]
(0.433779686463009-0.09419927821507568j)
[1.19294236-0.0888605j]
(0.4290050913753686-0.0512056456677662j)
[1.40309264-0.09038973j]
(0.4282624409805067-0.051035310572828334j)


In [15]:
current_model

[]